# report

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os
import copy
from contract_select import contract_select
from hedge_calc import hedge_calc
from greek_calc import greek_calc, pnl_daily_calc
from position_adj_cost import position_adj_cost

In [19]:
moneyness = 'out'
main_type = 'P'
hedge_type = 'P'
main_minDistance = 0.08
hedge_minDistance = 0.06
main_original = contract_select(select_type='main', date= '2018-12-10', spy_spx='spx', contract_type = main_type, minAsk=0.2, minDistance = main_minDistance, minDTE=15, moneyness=moneyness, num=3)
hedge_original = contract_select(select_type='hedge', date= '2018-12-10', spy_spx='spx', contract_type = hedge_type, minAsk=0.2, minDistance = hedge_minDistance, minDTE=15, moneyness=moneyness, num=3)
main_original.update(hedge_original)

position = hedge_calc(main_original)

# calculate the initial position spread cost
position_initial = copy.deepcopy(position)
position_initial['main']['WEIGHT'] = position_initial['main']['WEIGHT'] * 2
position_initial['hedge']['WEIGHT'] = position_initial['hedge']['WEIGHT'] * 2
position_initial['emini']['WEIGHT'] = position_initial['emini']['WEIGHT'] * 2
initial_spread_cost = position_adj_cost(position, position_initial)['adjust_cost']

greek_pnl_daily = pd.DataFrame()
greek_pnl_daily['daily_pnl'] = 0
greek_pnl_daily['adjust_cost'] = 0

d = copy.deepcopy(position)
daily = []
for i in range(250):
    greek_pnl_daily = greek_pnl_daily.append(greek_calc(d))
    d = pnl_daily_calc(d)
    greek_pnl_daily['daily_pnl'][-1] =  d['daily_rtn']
    daily.append(d)
    # print(d['main'].index[0][0], d['main']['[DTE]'][0], d['daily_rtn'])
    temp = copy.deepcopy(hedge_calc(d))
    
    # drop/wind up certain contracts that meet some criteria
    date = temp['main'].index[0][0]
    if main_type == 'P': temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] > temp['main']['[STRIKE]']]
    if main_type == 'C': temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] < temp['main']['[STRIKE]']]
    temp['main'] = temp['main'][temp['main']['[DTE]'] > 9]
    temp['main'] = temp['main'][temp['main']['[STRIKE_DISTANCE_PCT]'] >= main_minDistance/2]
    if len(temp['main']) < 3:
        new_contract = contract_select(select_type='main', date = date, spy_spx='spx', contract_type = main_type, minAsk=0.2, minDistance = main_minDistance, minDTE=15, moneyness=moneyness, num=3-len(temp['main']))
        temp['main'] = temp['main'].append(new_contract['main'])
        temp['main']['WEIGHT'] = -1/len(temp['main'])
    
    if hedge_type == 'P': temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] > temp['hedge']['[STRIKE]']]
    if hedge_type == 'C': temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] < temp['hedge']['[STRIKE]']]
    temp['hedge'] = temp['hedge'][temp['hedge']['[DTE]'] > 9]
    temp['hedge'] = temp['hedge'][temp['hedge']['[STRIKE_DISTANCE_PCT]'] >= hedge_minDistance/2]
    if len(temp['hedge']) < 3:
        new_contract = contract_select(select_type='hedge', date = date, spy_spx='spx', contract_type = hedge_type, minAsk=0.2, minDistance = hedge_minDistance, minDTE=15, moneyness=moneyness, num=3-len(temp['hedge']))
        temp['hedge'] = temp['hedge'].append(new_contract['hedge'])
        temp['hedge']['WEIGHT'] = 1/len(temp['hedge'])
        
    greek_pnl_daily['adjust_cost'][-1] = position_adj_cost(d, temp)['adjust_cost']
    d=copy.deepcopy(temp)

In [20]:
greek_pnl_daily.to_clipboard()

In [26]:
daily[19]['hedge']

[UNDERLYING_LAST]  [DTE]  [STRIKE]  [P_BID]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                
2019-01-09   2019-01-18               2585.08    9.0    2270.0     0.26   
             2019-01-18               2585.08    9.0    2265.0     0.25   
             2019-01-18               2585.08    9.0    2260.0     0.20   

                            [P_ASK]  [P_DELTA]  [P_GAMMA]  [P_VEGA]  \
[QUOTE_DATE] [EXPIRE_DATE]                                            
2019-01-09   2019-01-18        0.35   -0.00667    0.00011   0.07999   
             2019-01-18        0.34   -0.00654    0.00009   0.07963   
             2019-01-18        0.35   -0.00580    0.00014   0.07404   

                            [P_THETA]  [STRIKE_DISTANCE]  \
[QUOTE_DATE] [EXPIRE_DATE]                                 
2019-01-09   2019-01-18      -0.11327              315.1   
             2019-01-18      -0.11388              320.1   
             2019-01-18      -0.10574              325.1   

                            [STRIKE_DISTANCE_PCT]    WEIGHT   PNL  \
[QUOTE_DATE] [EXPIRE_DATE]                                          
2019-01-09   2019-01-18                     0.122  0.679302 -0.20   
             2019-01-18                     0.124 -0.320694 -0.26   
             2019-01-18                     0.126 -0.000002 -0.20   

                            PNL_PERCENTAGE  
[QUOTE_DATE] [EXPIRE_DATE]                  
2019-01-09   2019-01-18      -6.376907e-03  
             2019-01-18       3.913647e-03  
             2019-01-18       2.194092e-08